In [1]:
import pandas as pd

### 1) JOIN, aggregates, aliases, WHERE, GROUP BY, ORDER BY
Creates a query that lists each movie, the film category it is classified in, and the number of times it has been rented out.

``` mysql
SELECT f.title, c.name AS category, COUNT(r.rental_id) AS rental_count
FROM category c
JOIN film_category fc
ON c.category_id = fc.category_id
JOIN film f
ON f.film_id = fc.film_id
JOIN inventory i
ON f.film_id = i.film_id
JOIN rental r
ON i.inventory_id = r.inventory_id
WHERE c.name IN ('Animation', 'Children', 'Classics', 'Comedy', 'Family', 'Music')
GROUP BY 1, 2
ORDER BY 2, 1
```

### 2) window function, NTILE, IN
Provides a table with the movie titles and divide them into 4 levels (first_quarter, second_quarter, third_quarter, and final_quarter) based on the quartiles (25%, 50% 75%) of the rental duration for movies across all categories.
``` mysql
SELECT f.title,
       c.name AS category,
       f.rental_duration,
       NTILE(4) OVER(ORDER BY f.rental_duration) AS quartile
FROM category c
JOIN film_category fc
ON c.category_id = fc.category_id
JOIN film f
ON f.film_id = fc.film_id
WHERE c.name IN ('Animation', 'Children', 'Classics', 'Comedy', 'Family', 'Music')
```

### 3) common table expressions (CTEs)
Returns a table with the family-friendly film category, each of the quartiles, and the corresponding count of movies within each combination of film category for each corresponding rental duration category.
``` mysql
WITH cat_quart AS 
    (SELECT f.title, c.name AS category, f.rental_duration, 
            NTILE(4) OVER(ORDER BY f.rental_duration) AS quartile
     FROM category c
     JOIN film_category fc
     ON c.category_id = fc.category_id
     JOIN film f
     ON f.film_id = fc.film_id
     WHERE c.name IN ('Animation', 'Children', 'Classics', 
                       'Comedy', 'Family', 'Music'))

SELECT category, quartile, COUNT(*)
FROM cat_quart
GROUP BY 1, 2
ORDER BY 1, 2
```

### 4) sub-queries
**Inquiry 1**: How do the total number of rentals for each family-friendly genre compare to the average total number of rentals across all genres?

*Show each family friendly genre and the number of rental instances for each.*
``` mysql
SELECT category, SUM(rental_count) AS total_rentals
FROM (SELECT f.title, c.name AS category, COUNT(r.rental_id) AS rental_count
      FROM category c
      JOIN film_category fc
      ON c.category_id = fc.category_id
      JOIN film f
      ON f.film_id = fc.film_id
      JOIN inventory i
      ON f.film_id = i.film_id
      JOIN rental r
      ON i.inventory_id = r.inventory_id
      WHERE c.name IN ('Animation', 'Children', 'Classics', 'Comedy', 'Family', 'Music')
      GROUP BY 1, 2) fam_movies
GROUP BY 1
```

*Show average total rentals across all genres.*
``` mysql
WITH base AS 
    (SELECT category, SUM(rental_count) AS total_rentals
     FROM (SELECT f.title, c.name AS category, COUNT(r.rental_id) AS rental_count
           FROM category c
           JOIN film_category fc
           ON c.category_id = fc.category_id
           JOIN film f
           ON f.film_id = fc.film_id
           JOIN inventory i
           ON f.film_id = i.film_id
           JOIN rental r
           ON i.inventory_id = r.inventory_id
           GROUP BY 1, 2) fam_movies
     GROUP BY 1)
SELECT AVG(total_rentals)
FROM base
```

### 5) multiple CTEs, running total, UNION, DATE_TRUNC
**Inquiry 2**: All payment dates are from 2007. How do the two stores’ running revenues compare over the 2007 year?

*I use two CTEs to create a table for each store showing the total revenue for each day. I then use UNION ALL and a window to combine the tables and show the running total revenue by
store ordered by day. I wanted to do this with one query, paritioning by ID and ordering by day, but couldn't figure out how to combine DATE_TRUNC and OVER to get the results I wanted.*
``` mysql
WITH s1_dayrev AS
    (SELECT
        DATE_TRUNC('day', p.payment_date) AS day,
        s.store_id AS store,
        SUM(p.amount) AS day_total
    FROM payment p
    JOIN staff st
    ON st.staff_id = p.staff_id
    JOIN store s
    ON st.store_id = s.store_id
    WHERE s.store_id = 1
    GROUP BY 1,2),

	s2_dayrev AS
    (SELECT
        DATE_TRUNC('day', p.payment_date) AS day,
        s.store_id AS store,
        SUM(p.amount) AS day_total
    FROM payment p
    JOIN staff st
    ON st.staff_id = p.staff_id
    JOIN store s
    ON st.store_id = s.store_id
    WHERE s.store_id = 2
    GROUP BY 1, 2)

SELECT day, store, day_total,
	SUM(day_total) OVER (ORDER BY day) AS running_total_revenue
FROM s1_dayrev

UNION ALL

SELECT day, store, day_total,
	SUM(day_total) OVER (ORDER BY day) AS running_total_revenue
FROM s2_dayrev
```

### 6) CASE
**Inquiry**: How are the customers based in Europe, Asia, and Africa distributed
by country?

*Categorize countries into "low," "medium," "high," and "very high" customer count levels using a CASE clause and GROUP BY.*
``` mysql
SELECT co.country,
	  COUNT(customer_id) AS customer_ct,
      CASE WHEN COUNT(customer_id) < 15 THEN 'low'
           WHEN COUNT(customer_id) < 30 THEN 'medium'
           WHEN COUNT(customer_id) < 45 THEN 'high'
           ELSE 'very high' END AS customer_ct_level
FROM country co
JOIN city ci
ON co.country_id = ci.country_id
JOIN address a
ON ci.city_id = a.city_id
JOIN customer cu
ON a.address_id = cu.address_id
GROUP BY 1
ORDER BY 2 DESC
```

### 7) LEFT JOIN, CONCAT, DISTINCT
**Inquiry**: Is there a correlation between how prolific an actor is and the
number of times the movies they are in get rented?

*Here I concatenated the first and last names of each actor, counted the number of films they have been in, and counted the total number of times a movie with each actor had been rented. To be clear, the third column is the result of taking every movie actor X has been in and summing the number of times each movie in that set has been rented.*
``` mysql
SELECT a.first_name||' '||a.last_name AS actor,
	   COUNT (DISTINCT f.film_id) AS number_of_films,
       COUNT(r.rental_id) AS number_of_rents
FROM actor a
LEFT JOIN film_actor fa
ON a.actor_id = fa.actor_id
JOIN film f
ON f.film_id = fa.film_id
JOIN inventory i
ON f.film_id = i.film_id
JOIN rental r
ON i.inventory_id = r.inventory_id
GROUP BY 1
ORDER BY 2 DESC
```